In [151]:
from casadi import *
from time import time
import numpy as np
from tabulate import tabulate
#from simulation_code import simulate

# Paper Cascaded Nonlinear MPC for Realtime Quadrotor Position Tracking. Schlagenhauf et al.
# Non-Linear Model Predictive Control Using CasADi Package for Trajectory Tracking of Quadrotor. Elhesasy et al.

# Define system, static/kinematic/dynamic model. Typically Differential (Algebraic) equations

# Expression graphs represent a computation in the computer memory. SX, MX


Ix = 0.0000166  #Moment of inertia around p_WB_W_x-axis, source: Julian Förster's ETH Bachelor Thesis
Iy = 0.0000167  #Moment of inertia around p_WB_W_y-axis, source: Julian Förster's ETH Bachelor Thesis
Iz = 0.00000293 #Moment of inertia around p_WB_W_z-axis, source: Julian Förster's ETH Bachelor Thesis
m = 0.029 #mass of Crazyflie 2.1
g = 9.81

Nx = 12
Nu = 4
Nhoriz = 10
#T = 5. # time horizon

# x(t) = [x, y, z, phi, theta, psi, x_dot, y_dot, z_dot, phi_dot, ˙theta_dot, psi_dot]T
x = MX.sym("x", Nx)
u = MX.sym("u", Nu)


x_pos = x[0]  # x-position
y = x[1]  # y-position
z = x[2]  # z-position
phi = x[3]  # phi-angle, Euler angles
theta = x[4]  # theta-angle, Euler angles
psi = x[5]  # psi-angle, Euler angles
x_pos_dot = x[6]  # x velocity
y_dot = x[7]  # y velocity
z_dot = x[8]  # z velocity
phi_dot = x[9]  # phi_dot, angular velocity
theta_dot = x[10]  # theta_dot
psi_dot = x[11]  # psi-dot

thrust = u[0]
tau_phi = u[1]
tau_theta = u[2]
tau_psi = u[3]

x_pos_init = 0
y_init = 0
z_init = 0
phi_init = 0 #roll
theta_init = 0 #pitch
psi_init = 0 # yaw
x_pos_dot_init = 0
y_dot_init = 0
z_dot_init = 0
phi_dot_init = 0
theta_dot_init = 0
psi_dot_init = 0


x_pos_target = 0.1
y_target = 0.1
z_target = 0.1
phi_target = 0 #roll
theta_target = 0 #pitch
psi_target = 0.3 #yaw
x_pos_dot_target = 0
y_dot_target = 0
z_dot_target = 0
phi_dot_target = 0
theta_dot_target = 0
psi_dot_target = 0


# x˙(t) = [x_dot, y_dot, z_dot, phi_dot, ˙theta_dot, psi_dot, x_ddot, y_ddot, z_ddot, phi_ddot, ˙theta_ddot, psi_ddot]T

# Drag coefficients
K1 = 0.1
K2 = 0.1
K3 = 0.1


# Non-Linear Model Predictive Control Using CasADi Package for Trajectory Tracking of Quadrotor. Elhesasy et al.
#x_pos_ddot = (cos(phi) * sin(theta) * cos(psi) + sin(phi) * sin(psi))  * thrust / m 
#y_ddot = (cos(phi) * sin(theta) * cos(psi) - sin(phi) * cos(psi))  * thrust / m 
#z_ddot = -g + (cos(phi) * cos(theta)) * thrust / m 

# Novel Lyapunov-Based Autonomous Controllers for Quadrotors. Raj et al.
x_pos_ddot = (cos(phi) * sin(theta) * cos(psi) + sin(phi) * sin(psi))  * thrust / m - K1/m * x_pos_dot 
y_ddot = (cos(phi) * sin(theta) * cos(psi) - sin(phi) * cos(psi))  * thrust / m - K2/m * y_dot 
z_ddot = -g + (cos(phi) * cos(theta)) * thrust / m - K3/m * z_dot 

phi_ddot = theta_dot * psi_dot * (Iy - Iz) / (Ix) + tau_phi / Ix
theta_ddot = phi_dot * psi_dot * (Iz - Ix) / (Iy) + tau_theta / Iy
psi_ddot = theta_dot * phi_dot * (Ix - Iy) / (Iz) + tau_psi / Iz

x_dot = vertcat(x_pos_dot, y_dot, z_dot, phi_dot, theta_dot, psi_dot, x_pos_ddot, y_ddot, z_ddot, phi_ddot, theta_ddot, psi_ddot)

f = Function('f', [x, u], [x_dot], ['x', 'u'], ['x_dot'])

f

Function(f:(x[12],u[4])->(x_dot[12]) MXFunction)

In [152]:
### Testing the dynamics function

In [153]:
def run_dynamics_test(x_test_values, u_test_values, expected_x_dot_values):
    x_dot_test = f(x=x_test, u=u_test)
    x_dot_test_np = np.array(x_dot_test['x_dot']).flatten()
    
    try:
        np.testing.assert_almost_equal(x_dot_test_np, expected_x_dot_values, decimal=5)
        print("Test Passed")
    except AssertionError as e:
        print("Test Failed:", e)

In [154]:
#                 x  y  z   phi tht psi  x. y. z. phi. tht. psi.
x_test = vertcat([0, 0, 0,   0, 0,  0,   0, 0, 0,  0,   0,  0])

#               thrust tphi ttht tpsi
u_test = vertcat([m * g, 0,  0,   0])

#                          x  y  z  phi tht psi  x. y. z. phi. tht. psi.
expected_x_dot = np.array([0, 0, 0,   0, 0, 0,   0, 0, 0,    0, 0, 0])
#print(f(x=x_test, u=u_test))
run_dynamics_test(x_test, u_test, expected_x_dot)

Test Passed


In [155]:
#                 x  y  z   phi tht psi  x. y. z. phi. tht. psi.
x_test = vertcat([0, 0, 0,   0, 0,  0,   0, 0, 0,  0,   0,  0])

#               thrust tphi ttht tpsi
u_test = vertcat([0, 0,  0,   0])

#                          x  y  z  phi tht psi  x. y. z.   phi. tht. psi.
expected_x_dot = np.array([0, 0, 0,  0, 0, 0,    0, 0, -9.81,  0, 0, 0])
#print(f(x=x_test, u=u_test))
run_dynamics_test(x_test, u_test, expected_x_dot)

Test Passed


In [156]:
#                 x  y  z   phi tht psi  x. y. z. phi. tht. psi.
x_test = vertcat([0, 0, 0,   0, 0,  0,   0.1, 0, 0,  0,   0,  0])

#               thrust tphi ttht tpsi
u_test = vertcat([m * g, 0,  0,   0])

#                          x  y  z phi tht psi x. y. z. phi. tht. psi.
expected_x_dot = np.array([0.1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
print(f(x=x_test, u=u_test))
#run_dynamics_test(x_test, u_test, expected_x_dot)

{'x_dot': DM([0.1, 0, 0, 0, 0, 0, -0.344828, 0, 0, 0, 0, 0])}


In [157]:
#                 x  y  z   phi tht psi  x. y. z. phi. tht. psi.
x_test = vertcat([0, 0, 0,   0, 0,  0,   0, 0.1, 0,  0,   0,  0])

#               thrust tphi ttht tpsi
u_test = vertcat([m * g, 0,  0,   0])

#                          x  y  z phi tht psi x. y. z. phi. tht. psi.
expected_x_dot = np.array([0, 0.1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
#print(f(x=x_test, u=u_test))
run_dynamics_test(x_test, u_test, expected_x_dot)

Test Failed: 
Arrays are not almost equal to 5 decimals

Mismatched elements: 1 / 12 (8.33%)
Max absolute difference: 0.3448
Max relative difference: 0.
 x: array([ 0.     ,  0.1    ,  0.     ,  0.     ,  0.     ,  0.     ,
        0.     , -0.34483,  0.     ,  0.     ,  0.     ,  0.     ])
 y: array([0. , 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ])


In [158]:
#                 x  y  z   phi tht psi  x. y. z. phi. tht. psi.
x_test = vertcat([0, 0, 0,   0, 0,  0,   0, 0, 0.1,  0,   0,  0])

#               thrust tphi ttht tpsi
u_test = vertcat([m * g, 0,  0,   0])

#                          x  y  z phi tht psi x. y. z. phi. tht. psi.
expected_x_dot = np.array([0, 0, 0.1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
#print(f(x=x_test, u=u_test))
run_dynamics_test(x_test, u_test, expected_x_dot)

Test Failed: 
Arrays are not almost equal to 5 decimals

Mismatched elements: 1 / 12 (8.33%)
Max absolute difference: 0.3448
Max relative difference: 0.
 x: array([ 0.     ,  0.     ,  0.1    ,  0.     ,  0.     ,  0.     ,
        0.     ,  0.     , -0.34483,  0.     ,  0.     ,  0.     ])
 y: array([0. , 0. , 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ])


In [159]:
#                 x  y  z   phi tht psi  x. y. z. phi. tht. psi.
x_test = vertcat([0, 0, 0,   0, 0,  0,   0.1, 0.1, 0.1,  0,   0,  0])

#               thrust tphi ttht tpsi
u_test = vertcat([m * g, 0,  0,   0])

#                          x  y  z phi tht psi x. y. z. phi. tht. psi.
expected_x_dot = np.array([0.1, 0.1, 0.1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
#print(f(x=x_test, u=u_test))
run_dynamics_test(x_test, u_test, expected_x_dot)

Test Failed: 
Arrays are not almost equal to 5 decimals

Mismatched elements: 3 / 12 (25%)
Max absolute difference: 0.3448
Max relative difference: 0.
 x: array([ 0.1    ,  0.1    ,  0.1    ,  0.     ,  0.     ,  0.     ,
       -0.34483, -0.34483, -0.34483,  0.     ,  0.     ,  0.     ])
 y: array([0.1, 0.1, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ])


In [160]:
#                 x  y  z   phi tht psi  x. y. z. phi. tht. psi.
x_test = vertcat([0, 0, 0,   0.1, 0,  0,   0, 0, 0,  0,   0,  0])

#               thrust tphi ttht tpsi
u_test = vertcat([m * g, 0,  0,   0])

#                          x  y  z phi tht psi x. y. z. phi. tht. psi.
expected_x_dot = np.array([0, 0.1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
print(f(x=x_test, u=u_test))
#run_dynamics_test(x_test, u_test, expected_x_dot)

{'x_dot': DM([0, 0, 0, 0, 0, 0, 0, -0.979366, -0.0490091, 0, 0, 0])}


In [161]:
U = MX.sym("U", Nu, Nhoriz) # Decision variables (controls)
P = MX.sym('P',Nx + Nx) #parameters (which include the initial state and the reference state)
X = MX.sym("X", Nx, Nhoriz + 1) # A vector that represents the states over the optimization problem.

In [162]:
J = 0 # Objective function
g = []  # constraints vector

#Q=diag(MX([100, 100, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
Q = diagcat(100, 100, 100, 1, 1, 1, 1, 1, 1, 1, 1, 1)
#R=diag(MX([10.0, 10.0, 10.0, 10.0]))
R = diagcat(10.0, 10.0, 10.0, 10.0)



x_init = P[0:Nx]
g = X[:, 0] - P[0:Nx]  # initial condition constraints
h = 0.2
J = 0

#u_test_np = np.array([m * g, 0,   0,   0]) 

for k in range(Nhoriz-1):  
    st_ref = P[Nx:2*Nx]
    st = X[:, k]
    cont = U[:, k]
    J += (st - st_ref).T @ Q @ (st - st_ref) + cont.T @ R @ cont
    st_next = X[:, k+1]
    k1 = f(st, cont)  
    k2 = f(st + h / 2 * k1, cont)
    k3 = f(st + h / 2 * k2, cont)
    k4 = f(st + h * k3, cont)
    st_next_RK4 = st + h/6*(k1 + 2*k2 + 2*k3 + k4)  # RK4 integration
    g = vertcat(g, st_next - st_next_RK4) # Multiple Shooting





In [163]:
OPT_variables = vertcat(
    X.reshape((-1, 1)),   # Example: 3x11 ---> 33x1 where 3=states, 11=N+1
    U.reshape((-1, 1))
)

nlp_prob = {
    'f': J,
    'x': OPT_variables,
    'g': g,
    'p': P
}

opts = {
    'ipopt': {
        'max_iter': 2000,
        'print_level': 0,
        'acceptable_tol': 1e-8,
        'acceptable_obj_change_tol': 1e-6
    },
    'print_time': 0
}

solver = nlpsol('solver', 'ipopt', nlp_prob, opts)

lbx = DM.zeros((Nx*(Nhoriz+1) + Nu*Nhoriz, 1))
ubx = DM.zeros((Nx*(Nhoriz+1) + Nu*Nhoriz, 1))

lbx[0: Nx*(Nhoriz+1): Nx] = -inf     # x lower bound
lbx[1: Nx*(Nhoriz+1): Nx] = -inf     # y lower bound
lbx[2: Nx*(Nhoriz+1): Nx] = -inf     # z lower bound
lbx[3: Nx*(Nhoriz+1): Nx] = -inf     # phi lower bound
lbx[4: Nx*(Nhoriz+1): Nx] = -inf     # theta lower bound
lbx[5: Nx*(Nhoriz+1): Nx] = -inf     # psi lower bound
lbx[6: Nx*(Nhoriz+1): Nx] = -inf     # x_dot lower bound
lbx[7: Nx*(Nhoriz+1): Nx] = -inf     # y_dot lower bound
lbx[8: Nx*(Nhoriz+1): Nx] = -inf     # z_dot lower bound
lbx[9: Nx*(Nhoriz+1): Nx] = -inf     # phi_dot lower bound
lbx[10: Nx*(Nhoriz+1): Nx] = -inf     # theta_dot lower bound
lbx[11: Nx*(Nhoriz+1): Nx] = -inf     # psi_dot lower bound


ubx[0: Nx*(Nhoriz+1): Nx] = inf     # x upper bound
ubx[1: Nx*(Nhoriz+1): Nx] = inf     # y upper bound
ubx[2: Nx*(Nhoriz+1): Nx] = inf     # z upper bound
ubx[3: Nx*(Nhoriz+1): Nx] = inf     # phi upper bound
ubx[4: Nx*(Nhoriz+1): Nx] = inf     # theta upper bound
ubx[5: Nx*(Nhoriz+1): Nx] = inf     # psi upper bound
ubx[6: Nx*(Nhoriz+1): Nx] = inf     # x_dot upper bound
ubx[7: Nx*(Nhoriz+1): Nx] = inf     # y_dot upper bound
ubx[8: Nx*(Nhoriz+1): Nx] = inf     # z_dot upper bound
ubx[9: Nx*(Nhoriz+1): Nx] = inf     # phi_dot upper bound
ubx[10: Nx*(Nhoriz+1): Nx] = inf     # theta_dot upper bound
ubx[11: Nx*(Nhoriz+1): Nx] = inf     # psi_dot upper bound

# Non-Linear Model Predictive Control Using CasADi Package for Trajectory Tracking of Quadrotor. Elhesasy et al.

'''
lbx[0: Nx*(Nhoriz+1):] = 0              # lower bound for Thrust 
lbx[1: Nx*(Nhoriz+1):] = -1.257               # lower bound for Tau_x 
lbx[2: Nx*(Nhoriz+1):] = -1.257               # lower bound for Tau_y 
lbx[3: Nx*(Nhoriz+1):] = -0.2145              # lower bound for Tau_z 

ubx[0: Nx*(Nhoriz+1):] = 36.257                  # upper bound for Thrust
ubx[1: Nx*(Nhoriz+1):] = 1.257                   # upper bound for Tau_x
ubx[2: Nx*(Nhoriz+1):] = 1.257                  # upper bound for Tau_y
ubx[3: Nx*(Nhoriz+1):] = 0.2145                  # upper bound for Tau_z
'''


lbx[0: Nx*(Nhoriz+1):] = -inf              # lower bound for Thrust 
lbx[1: Nx*(Nhoriz+1):] = -inf              # lower bound for Tau_x 
lbx[2: Nx*(Nhoriz+1):] = -inf              # lower bound for Tau_y 
lbx[3: Nx*(Nhoriz+1):] = -inf              # lower bound for Tau_z 

ubx[0: Nx*(Nhoriz+1):] = inf                # upper bound for Thrust
ubx[1: Nx*(Nhoriz+1):] = inf                # upper bound for Tau_x
ubx[2: Nx*(Nhoriz+1):] = inf                # upper bound for Tau_y
ubx[3: Nx*(Nhoriz+1):] = inf                # upper bound for Tau_z


args = {
    'lbg': DM.zeros((Nx*Nhoriz, 1)),  # constraints lower bound
    'ubg': DM.zeros((Nx*Nhoriz, 1)),  # constraints upper bound
    'lbx': lbx,
    'ubx': ubx
}

#   'lbg': DM.zeros((Nx*(Nhoriz+1), 1)),  # constraints lower bound
#   'ubg': DM.zeros((Nx*(Nhoriz+1), 1)),  # constraints upper bound


In [164]:
t0 = 0
state_init = DM([x_pos_init, y_init, z_init, phi_init, theta_init, psi_init, x_pos_dot_init, y_dot_init, z_dot_init, phi_dot_init, theta_dot_init, psi_dot_init])        # initial state
state_target = DM([x_pos_target, y_target, z_target, phi_target, theta_target, psi_target, x_pos_dot_target, y_dot_target, z_dot_target, phi_dot_target, theta_dot_target, psi_dot_target])  # target state

# xx = DM(state_init)
t = DM(t0)

u0 = DM.zeros((Nu, Nhoriz))  # initial control
X0 = repmat(state_init, 1, Nhoriz+1)         # initial state full


def DM2Arr(dm):
    return np.array(dm.full())

mpc_iter = 0
cat_states = DM2Arr(X0)
cat_controls = DM2Arr(u0[:, 0])
times = np.array([[0]])

In [165]:
def shift_timestep(step_horizon, t0, state_init, u, f):
    f_value = f(state_init, u[:, 0])
    next_state = DM.full(state_init + (step_horizon * f_value))

    t0 = t0 + step_horizon
    u0 = horzcat(
        u[:, 1:],
        reshape(u[:, -1], -1, 1)
    )

    return t0, next_state, u0

In [166]:
main_loop = time()  # return time in sec
while (norm_2(state_init - state_target) > 1e-2) and (mpc_iter * h < 200):
    t1 = time()
    args['p'] = vertcat(
        state_init,    # current state
        state_target   # target state
    )
    # optimization variable current state
    args['x0'] = vertcat(
        reshape(X0, Nx*(Nhoriz+1), 1),
        reshape(u0, Nu*Nhoriz, 1)
    )

    sol = solver(
        x0=args['x0'],
        lbx=args['lbx'],
        ubx=args['ubx'],
        lbg=args['lbg'],
        ubg=args['ubg'],
        p=args['p']
    )

    u = reshape(sol['x'][Nx * (Nhoriz + 1):], Nu, Nhoriz)
    
    u_np = np.array(u.full())
    np.set_printoptions(precision=4, suppress=True)
    #print(u_np)
    
    print("Control Inputs (u) at this iteration:")
    print(tabulate(u_np, tablefmt="fancy_grid", showindex="always", headers=[f"Step {i}" for i in range(Nhoriz)]))

    
    X0 = reshape(sol['x'][: Nx * (Nhoriz+1)], Nx, Nhoriz+1)
    
    

    cat_states = np.dstack((
        cat_states,
        DM2Arr(X0)
    ))

    cat_controls = np.vstack((
        cat_controls,
        DM2Arr(u[:, 0])
    ))
    t = np.vstack((
        t,
        t0
    ))

    t0, state_init, u0 = shift_timestep(h, t0, state_init, u, f)

    # print(X0)
    X0 = horzcat(
        X0[:, 1:],
        reshape(X0[:, -1], -1, 1)
    )

    # xx ...
    t2 = time()
    print(mpc_iter)
    print(t2-t1)
    times = np.vstack((
        times,
        t2-t1
    ))

    mpc_iter = mpc_iter + 1

main_loop_time = time()
ss_error = norm_2(state_init - state_target)

print('\n\n')
print('Total time: ', main_loop_time - main_loop)
print('avg iteration time: ', np.array(times).mean() * 1000, 'ms')
print('final error: ', ss_error)

# simulate
# simulate(cat_states, cat_controls, times, h, Nhoriz,
#         np.array([x_init, y_init, theta_init, x_target, y_target, theta_target]), save=False)

Control Inputs (u) at this iteration:
╒════╤══════════╤══════════╤══════════╤══════════╤══════════╤══════════╤══════════╤══════════╤══════════╤══════════╕
│    │   Step 0 │   Step 1 │   Step 2 │   Step 3 │   Step 4 │   Step 5 │   Step 6 │   Step 7 │   Step 8 │   Step 9 │
╞════╪══════════╪══════════╪══════════╪══════════╪══════════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│  0 │        0 │        0 │        0 │        0 │        0 │        0 │        0 │        0 │        0 │        0 │
├────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│  1 │        0 │        0 │        0 │        0 │        0 │        0 │        0 │        0 │        0 │        0 │
├────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┤
│  2 │        0 │        0 │        0 │        0 │        0 │        0 │        0 │        0 │        0 │        0 │
├────┼──────────┼─────────

CasADi - 2024-02-02 15:33:10 WARNING("KeyboardInterruptException") [.../casadi/interfaces/ipopt/ipopt_nlp.cpp:114]


SystemError: <built-in function Function_call> returned a result with an exception set